In [1]:
from dependencies.combine_frontpage_gre import CombineFrontpageGRE
import pandas as pd
import re
import os
import datetime
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

def get_date_from_filepath(filepath):
    date_match = re.search(r'(\d{8})', filepath)

    if date_match:
        date_str = date_match.group(1)
        print(f"Extracted date string: {date_str}")
        # Parse the date string and format it
        date_obj = datetime.datetime.strptime(date_str, '%d%m%Y')
        previous_week_date = date_obj - datetime.timedelta(days=7)
        formatted_previous_week_date = previous_week_date.strftime('%d%m%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date, formatted_previous_week_date, previous_week_date
    else:
        raise ValueError("No date found in the filename.")

base_directory = "/mnt/box/Risk Report/2024"
gre_data_filepath = '/mnt/box/Risk Report/Intern 2024/Firmwide Risk Report/GDLP GRE Data/GDLP GRE Data 02082024.csv'

# Initialize CombineFrontpageGRE
combine_processor = CombineFrontpageGRE(base_directory, gre_data_filepath)

# Run the combined process
combine_processor.run()

date, formatted_previous_week_date, previous_week_date = get_date_from_filepath(gre_data_filepath)

Extracted date string: 02082024
Extracted date string: 02082024
        Beimnet BTC Core Delta  Beimnet BTC Options Delta  Beimnet BTC Delta  \
Values            24350.374917              -1.370348e+06      -1.345997e+06   

        Beimnet ETH Core Delta  Beimnet ETH Delta  Beimnet Alts Core Delta  \
Values             1415.952957        1415.952957            336702.467712   

        Beimnet Alts Delta  Beimnet Delta  Bouchra BTC Core Delta  \
Values       336702.467712  -1.007879e+06            7.439207e+06   

        Bouchra BTC Options Delta  ...  GDLP BTC Total Gamma  \
Values               9.204192e+06  ...        -360226.012801   

        GDLP ETH Total Gamma  GDLP SOL Total Gamma  GDLP Alts Total Gamma  \
Values         -20129.808927                     0                      0   

        GDLP Total Gamma  GDLP BTC Total Vega  GDLP ETH Total Vega  \
Values                 0         28903.468739         -1923.341495   

        GDLP SOL Total Vega  GDLP Alts Total Vega  GDL

In [7]:
print(formatted_previous_week_date)

26072024


In [2]:
previous_week_gre_filepath = f'/mnt/box/Risk Report/Intern 2024/Firmwide Risk Report/GDLP GRE Data/GDLP GRE Data {formatted_previous_week_date}.csv'

combine_processor = CombineFrontpageGRE(base_directory, previous_week_gre_filepath)

combine_processor.run()

Extracted date string: 26072024
Extracted date string: 26072024
        Beimnet BTC Core Delta  Beimnet BTC Options Delta  Beimnet BTC Delta  \
Values            2.431191e+06               7.033446e+06       9.464637e+06   

        Beimnet ETH Core Delta  Beimnet ETH Delta  Beimnet SOL Core Delta  \
Values           -1.250177e+07      -1.250177e+07          -681100.646275   

        Beimnet SOL Delta  Beimnet Alts Core Delta  Beimnet Alts Delta  \
Values     -681100.646275            -970958.92184       -970958.92184   

        Beimnet Delta  ...  GDLP BTC Total Gamma  GDLP ETH Total Gamma  \
Values  -4.689197e+06  ...          3.823476e+06         -28317.701291   

        GDLP SOL Total Gamma  GDLP Alts Total Gamma  GDLP Total Gamma  \
Values                     0                      0                 0   

        GDLP BTC Total Vega  GDLP ETH Total Vega  GDLP SOL Total Vega  \
Values         10597.766073         -3355.315227                    0   

        GDLP Alts Total Vega

In [5]:
print(previous_week_date)

2024-07-26


In [3]:
previous_week_date = previous_week_date.strftime('%Y-%m-%d')
# Load the CSV data into a DataFrame
csv_filepath = f'../output/{date}.csv'
df = pd.read_csv(csv_filepath)

prev_csv_filepath = f'../output/{previous_week_date}.csv'
prev_df = pd.read_csv(prev_csv_filepath)

# Load the Excel template
template_path = '../data/Template.xlsx'
wb = load_workbook(template_path)

# Print the sheet names to find the correct one
print(wb.sheetnames)

# Replace 'CorrectSheetName' with the actual sheet name from the template
template_sheet_name = 'Sheet1'
template_sheet = wb[template_sheet_name]

# Get the sheet where the CSV data will be placed
new_sheet_name = 'CSV Data'
if new_sheet_name in wb.sheetnames:
    data_sheet = wb[new_sheet_name]
else:
    data_sheet = wb.create_sheet(title=new_sheet_name)

# Write the DataFrame to the new sheet
for r in dataframe_to_rows(df, index=False, header=True):
    data_sheet.append(r)

new_sheet_name_prev = 'Prev. Week CSV Data'
if new_sheet_name_prev in wb.sheetnames:
    prev_data_sheet = wb[new_sheet_name_prev]
else:
    prev_data_sheet = wb.create_sheet(title=new_sheet_name_prev)

for r in dataframe_to_rows(prev_df, index=False, header=True):
    prev_data_sheet.append(r)

# Define a mapping from CSV columns to template cells for Delta values
delta_mapping = {
    'Novo BTC Delta': 'C4',
    'GDLP ex-Novo BTC Delta': 'C5',
    'Novo ETH Delta': 'C8',
    'GDLP ex-Novo ETH Delta': 'C9',
    'Novo SOL Delta': 'C12',
    'GDLP ex-Novo SOL Delta': 'C13',
    'Novo Alts Delta': 'C16',
    'GDLP ex-Novo Alts Delta': 'C17',
    'Liquid Ventures Delta': 'C19',
    'Passive Beta Delta': 'C20'
}

# Define a mapping from CSV columns to template cells for Gamma values
gamma_mapping = {
    'Novo BTC Options Gamma': 'G4',
    'GDLP ex-Novo BTC Gamma': 'G5',
    'Novo ETH Options Gamma': 'G8',
    'GDLP ex-Novo ETH Gamma': 'G9',
    'Novo SOL Options Gamma': 'G12',
    'GDLP ex-Novo SOL Gamma': 'G13',
    'Novo Alts Gamma': 'G16',
    'GDLP ex-Novo Alts Gamma': 'G17',
}

# Define a mapping from CSV columns to template cells for Vega values
vega_mapping = {
    'Novo BTC Options Vega': 'K4',
    'GDLP ex-Novo BTC Vega': 'K5',
    'Novo ETH Options Vega': 'K8',
    'GDLP ex-Novo ETH Vega': 'K9',
    'Novo SOL Options Vega': 'K12',
    'GDLP ex-Novo SOL Vega': 'K13',
    'Novo Alts Vega': 'K16',
    'GDLP ex-Novo Alts Vega': 'K17',
}

price_mapping = {
    'BTC Price': 'Q3',
    'ETH Price': 'Q4',
    'SOL Price': 'Q5'
}

spot_mapping = {
    'Novo BTC Spot Delta': 'V4',
    'GDLP ex-Novo BTC Spot Delta': 'V5',
    'Novo ETH Spot Delta': 'V7',
    'GDLP ex-Novo ETH Spot Delta': 'V8',
    'Novo SOL Spot Delta': 'V10',
    'GDLP ex-Novo SOL Spot Delta': 'V11',
    'Novo Alts Spot Delta': 'V13',
    'GDLP ex-Novo Alts Spot Delta': 'V14'
}

options_mapping = {
    'Novo BTC Options Delta': 'W4',
    'GDLP ex-Novo BTC Options Delta': 'W5',
    'Novo ETH Options Delta': 'W7',
    'GDLP ex-Novo ETH Options Delta': 'W8',
    'Novo SOL Options Delta': 'W10',
    'GDLP ex-Novo SOL Options Delta': 'W11',
    'Novo Alts Options Delta': 'W13',
    'GDLP ex-Novo Alts Options Delta': 'W14'
}

prev_delta_mapping = {
    'Novo BTC Delta': 'D4',
    'GDLP ex-Novo BTC Delta': 'D5',
    'Novo ETH Delta': 'D8',
    'GDLP ex-Novo ETH Delta': 'D9',
    'Novo SOL Delta': 'D12',
    'GDLP ex-Novo SOL Delta': 'D13',
    'Novo Alts Delta': 'D16',
    'GDLP ex-Novo Alts Delta': 'D17',
    'Liquid Ventures Delta': 'D19',
    'Passive Beta Delta': 'D20'
}

prev_gamma_mapping = {
    'Novo BTC Options Gamma': 'H4',
    'GDLP ex-Novo BTC Gamma': 'H5',
    'Novo ETH Options Gamma': 'H8',
    'GDLP ex-Novo ETH Gamma': 'H9',
    'Novo SOL Options Gamma': 'H12',
    'GDLP ex-Novo SOL Gamma': 'H13',
    'Novo Alts Gamma': 'H16',
    'GDLP ex-Novo Alts Gamma': 'H17',
}

prev_vega_mapping = {
    'Novo BTC Options Vega': 'L4',
    'GDLP ex-Novo BTC Vega': 'L5',
    'Novo ETH Options Vega': 'L8',
    'GDLP ex-Novo ETH Vega': 'L9',
    'Novo SOL Options Vega': 'L12',
    'GDLP ex-Novo SOL Vega': 'L13',
    'Novo Alts Vega': 'L16',
    'GDLP ex-Novo Alts Vega': 'L17',
}

prev_price_mapping = {
    'BTC Price': 'R3',
    'ETH Price': 'R4',
    'SOL Price': 'R5'
}

for column, cell in delta_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

for column, cell in gamma_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

for column, cell in vega_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

for column, cell in price_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

for column, cell in spot_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

for column, cell in options_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

for column, cell in prev_delta_mapping.items():
    if column in prev_df.columns:
        value = prev_df[column].values[0]
        template_sheet[cell] = value

for column, cell in prev_gamma_mapping.items():
    if column in prev_df.columns:
        value = prev_df[column].values[0]
        template_sheet[cell] = value

for column, cell in prev_vega_mapping.items():
    if column in prev_df.columns:
        value = prev_df[column].values[0]
        template_sheet[cell] = value

for column, cell in prev_price_mapping.items():
    if column in prev_df.columns:
        value = prev_df[column].values[0]
        template_sheet[cell] = value

# # Also include {Trader} {Asset} Delta, Gamma, and Vega
# trader_asset_mappings = {
#     'Delta': delta_mapping,
#     'Gamma': gamma_mapping,
#     'Vega': vega_mapping
# }

# for trader in ['Beimnet', 'Bouchra', 'Eduardo', 'Felman']:
#     for asset in ['BTC', 'ETH', 'SOL']:
#         for greek, mapping in trader_asset_mappings.items():
#             column_name = f'{trader} {asset} {greek}'
#             if column_name in df.columns:
#                 cell = f"{chr(67 + len(mapping))}{4 + len(mapping)}"  # Adjust column index and row
#                 template_sheet[cell] = df[column_name].values[0]
#                 mapping[column_name] = cell

# Save the workbook with the filled-in data
output_path = f"../output/Firmwide Report {date}.xlsx"
wb.save(output_path)


['Sheet1']
